In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import datetime
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False

%matplotlib inline

In [2]:
colors = sns.color_palette()

In [3]:
file = r'../data/OnlineRetail.csv'
online = pd.read_csv(file, encoding='Unicode_escape', parse_dates=['InvoiceDate'])
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
mask = online['CustomerID'].isnull()
mask

0         False
1         False
2         False
3         False
4         False
          ...  
541904    False
541905    False
541906    False
541907    False
541908    False
Name: CustomerID, Length: 541909, dtype: bool

In [5]:
online_rfm = online[~mask]
online_rfm.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
online_rfm.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [7]:
online_rfm['Total'] = online_rfm['UnitPrice'] * online_rfm['Quantity']
online_rfm.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [11]:
current_date = max(online_rfm['InvoiceDate']) + datetime.timedelta(days=1)
current_date  # 假设这一天为当前日

Timestamp('2011-12-10 12:50:00')

In [22]:
df = online_rfm.groupby('CustomerID').agg({'InvoiceDate': lambda x:(current_date - x.max()).days,
                                          'InvoiceNo': 'count',
                                          'Total': 'sum'})
df.sample(3)

,InvoiceDate,InvoiceNo,Total
CustomerID,,,
12480.0,28,154,3281.63
15907.0,4,249,967.45
13548.0,96,124,682.41


In [23]:
df.rename(columns={'InvoiceDate':'Recency', 
                  'InvoiceNo': 'Frequency',
                  'Total': 'Monetary'}, inplace=True)

In [24]:
df

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.00
12347.0,2,182,4310.00
12348.0,75,31,1797.24
12349.0,19,73,1757.55
12350.0,310,17,334.40
...,...,...,...
18280.0,278,10,180.60
18281.0,181,7,80.82
18282.0,8,13,176.60


In [25]:
r = range(4, 0, -1)  # 因为距离越近越好,所以值越小,反而要更大
r_quartiles = pd.cut(df['Recency'], 4, labels=r)
r_quartiles

CustomerID
12346.0    1
12347.0    4
12348.0    4
12349.0    4
12350.0    1
          ..
18280.0    2
18281.0    3
18282.0    4
18283.0    4
18287.0    4
Name: Recency, Length: 4372, dtype: category
Categories (4, int64): [4 < 3 < 2 < 1]

In [27]:
f = range(1, 5)
m = range(1, 5)
f_quartiles = pd.qcut(df['Frequency'], 4, labels=f)
m_quartiles = pd.qcut(df['Monetary'], 4, labels=m)

In [40]:
df = df.assign(R=r_quartiles)
df = df.assign(F=f_quartiles)
df = df.assign(M=m_quartiles)

In [44]:
df

,Recency,Frequency,Monetary,R,F,M
CustomerID,,,,,,
12346.0,326,2,0.00,1,1,1
12347.0,2,182,4310.00,4,4,4
12348.0,75,31,1797.24,4,2,4
12349.0,19,73,1757.55,4,3,4
12350.0,310,17,334.40,1,1,2
...,...,...,...,...,...,...
18280.0,278,10,180.60,2,1,1
18281.0,181,7,80.82,3,1,1
18282.0,8,13,176.60,4,1,1


In [48]:
def concat_rfm(df):
    return str(df['R'])+str(df['F'])+str(df['M'])

df['RFM_Segment'] = df.apply(concat_rfm, axis=1)
df.sample(10)

,Recency,Frequency,Monetary,R,F,M,RFM_Segment
CustomerID,,,,,,,
16418.0,45,102,675.27,4,3,3,433
16976.0,267,2,251.52,2,1,1,211
17781.0,159,12,188.02,3,1,1,311
15146.0,165,95,1313.66,3,3,3,333
12680.0,1,52,862.81,4,3,3,433
17251.0,363,39,283.28,1,2,1,121
13685.0,3,39,3119.44,4,2,4,424
13448.0,17,199,3465.67,4,4,4,444
12707.0,292,33,603.42,1,2,2,122


In [52]:
def rfm_sum(df):
    return df['R']+df['F']+df['M']
df['RFM_score'] = df.apply(rfm_sum, axis=1)
df.sample(10)

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_score
CustomerID,,,,,,,,
15691.0,252,2,136.80,2,1,1,211,4
13947.0,65,5,90.60,4,1,1,411,6
15096.0,43,11,219.39,4,1,1,411,6
15932.0,4,119,2461.85,4,4,4,444,12
15576.0,38,45,940.67,4,3,3,433,10
13341.0,261,42,775.85,2,2,3,223,7
12518.0,1,124,2056.89,4,4,4,444,12
17596.0,23,270,2827.93,4,4,4,444,12
15069.0,179,70,1109.53,3,3,3,333,9


In [59]:
df.groupby('RFM_Segment').size().sort_values(ascending=False)[:10]

RFM_Segment
444    760
433    448
422    322
411    311
443    201
434    176
111    167
423    162
211    155
432    152
dtype: int64

In [70]:
def segment(df):
    if df['RFM_score'] >= 10:
        return 'Gold'
    elif df['RFM_score'] >= 6 and df['RFM_score'] <= 9:
        return 'Silver'
    else:
        return 'Bronze'

# def segment(series):
#     if series >= 10:
#         return 'Gold'
#     elif series >= 6 and series <= 9:
#         return 'Silver'
#     else:
#         return 'Bronze'
    
    
df['Segment'] = df.apply(segment, axis=1)
df.sample(5)

,Recency,Frequency,Monetary,R,F,M,RFM_Segment,RFM_score,Segment
CustomerID,,,,,,,,,
15897.0,194,24,222.05,2,2,1,221,5,Bronze
15669.0,40,35,677.69,4,2,3,423,9,Silver
14524.0,32,203,1867.68,4,4,4,444,12,Gold
16065.0,365,74,372.09,1,3,2,132,6,Silver
14890.0,254,7,125.65,2,1,1,211,4,Bronze


In [71]:
df.groupby('Segment').agg({'Recency': 'mean',
                           'Frequency': 'mean',
                           'Monetary': ['mean', 'count']}).round(1)

Recency Frequency Monetary      
           mean      mean     mean count
Segment                                 
Bronze    260.0      12.8    190.4   698
Gold       30.3     192.1   4053.8  1744
Silver     87.1      32.6    568.5  1930

In [72]:
df[df['Segment']=='Gold'].shape

(1744, 9)